In [3]:
def construit_liste_ss_arbres_caracteres_nombres(fichier, affiche = True):
    """Pour chaque caractère c du fichier, construit une liste :
    [(c,n), None, None] où n est le nombre de fois que c est présent dans le
    fichier. Une telle liste sera vue plus tard comme une feuille.
    Si affiche == True, afficher les paires (c,n) dans l'ordre croissant de n.
    """
    L = []
    dico = {}
    with open(fichier, "r", encoding="utf-8") as f:
        for line in f:
            for c in line:
                if c in dico:
                    dico[c] += 1
                else:
                    dico[c] = 1

    for i in dico.keys():
        l = []
        l.append((i, dico[i]))
        l.append(None)
        l.append(None)
        L.append(l)
        
    L.sort(key = lambda tup: tup[0][1])

    if affiche:
        print(L)
    return L


In [4]:
construit_liste_ss_arbres_caracteres_nombres('fichier_huffman.txt', affiche = True)

[[('V', 1), None, None], [('l', 1), None, None], [('t', 1), None, None], [('è', 1), None, None], [('à', 1), None, None], [(':', 1), None, None], [('!', 1), None, None], [('=', 1), None, None], [('2', 1), None, None], [('o', 2), None, None], [('i', 2), None, None], [('q', 2), None, None], [('u', 2), None, None], [('s', 2), None, None], [('d', 2), None, None], [('€', 2), None, None], [('4', 2), None, None], [('a', 3), None, None], [('r', 3), None, None], [('b', 3), None, None], [('e', 4), None, None], [('\n', 5), None, None], [('c', 7), None, None], [(' ', 8), None, None]]


[[('V', 1), None, None],
 [('l', 1), None, None],
 [('t', 1), None, None],
 [('è', 1), None, None],
 [('à', 1), None, None],
 [(':', 1), None, None],
 [('!', 1), None, None],
 [('=', 1), None, None],
 [('2', 1), None, None],
 [('o', 2), None, None],
 [('i', 2), None, None],
 [('q', 2), None, None],
 [('u', 2), None, None],
 [('s', 2), None, None],
 [('d', 2), None, None],
 [('€', 2), None, None],
 [('4', 2), None, None],
 [('a', 3), None, None],
 [('r', 3), None, None],
 [('b', 3), None, None],
 [('e', 4), None, None],
 [('\n', 5), None, None],
 [('c', 7), None, None],
 [(' ', 8), None, None]]

In [5]:
def construit_arbre_huffman_depuis_liste(liste_car_nbre):
    """À partir de la liste composée de listes du type [(c,n), None, None],
    construit et retourne l'arbre de Huffman suivant l'algorithme classique.
    Le résultat (l'arbre) est une liste composée de listes du type :
    [(c,n), a_1, a_2] avec :
    + n un entier.
    + c un caractère ; dans un cas a_1 et a_2 sont None et c'est une feuille
        ou c est None ; dans l'autre cas c'est un noeud interne et a_1 et a_2 sont
        des sous-arbres. Par convention, a_1 est le sous-arbre gauche codant 0
        et a_2 le sous-arbre droit codant 1."""
    arbre = liste_car_nbre
    while len(arbre) > 1:
        # Retirer les 2 noeuds ayant les plus petites valeurs n
        a_1 = min(arbre, key=lambda noeud: noeud[0][1])
        arbre.remove(a_1)
        a_2 = min(arbre, key=lambda noeud: noeud[0][1])
        arbre.remove(a_2)
        # a_1 et a_2 deviennent sous-arbres d'un nouveau noeud, somme des 2
        a_nouveau = (None, a_1[0][1] + a_2[0][1])
        arbre.append([a_nouveau, a_1, a_2])
    return arbre[0]


In [6]:
def construit_table_codage_depuis_arbre_huffman(arbre):
    """Construit la table de codage à partir de l'arbre de Huffman.
    Le resultat est un dictionnaire dont les clés sont les caractères et les
    valeurs sont les codes binaires correspondant issus de l'arbre. Un code
    binaire est retourné ici sous forme de chaine de cararctères de '0' et '1'.
    """
    def iter_rec_chaines_binaires(arbre, chaine_courante):
        if arbre[1] == None and arbre[2] == None: # feuille
            yield arbre[0][0], chaine_courante
        else:
            yield from iter_rec_chaines_binaires(arbre[1], chaine_courante + "0")
            yield from iter_rec_chaines_binaires(arbre[2], chaine_courante + "1")

    table = {}
    it = iter_rec_chaines_binaires(arbre, "")
    for (car, chaine) in it:
        table[car] = chaine
    return table

In [7]:
def code_fichier(fichier, table_codage):
    """Code chaque caractère du fichier avec la table de codage dont les clés
    sont les caractères et les valeurs les codes binaires sous forme de chaines
    de '0' et de '1'.
    Le résultat est une chaine de caractères de '0' et de '1'."""
    result = ""
    with open(fichier, "r", encoding="utf8") as f:
        for ligne in f:
            result += "".join([table_codage[car] for car in ligne])
    return result

In [8]:
def decode_message(message_binaire, arbre):
    """Prend en entrée une chaine de caractères de '0' et de '1' (message codé)
    + un arbre de huffman. Retourne le décodage sous forme d'une chaine de
    caractères."""
    result = ""
    sous_arbre = arbre
    for x in message_binaire:
        if x == '0':
            sous_arbre = sous_arbre[1]
            if sous_arbre[1] == None and sous_arbre[2] == None:
                result += sous_arbre[0][0]
                sous_arbre = arbre
        elif x == '1':
            sous_arbre = sous_arbre[2]
            if sous_arbre[1] == None and sous_arbre[2] == None:
                result += sous_arbre[0][0]
                sous_arbre = arbre
    return result

In [10]:
#----- Manipulations de ces fonctions.
# Partie codage du fichier :
fichier = "FICHIER_ESSAI_HUFFMAN.py"
#fichier = "Codage-Huffman-Simple.py" # Pour coder le fichier source lui-même.
liste_feuilles = construit_liste_ss_arbres_caracteres_nombres(fichier, False)
arbre = construit_arbre_huffman_depuis_liste(liste_feuilles)
table = construit_table_codage_depuis_arbre_huffman(arbre)
message_codé = code_fichier(fichier, table) # Codage Huffman en bin. du fichier 

print(f"Le message codé est :\n{message_codé}")
print(10*"---")
print(f"La taille du message codé est de : {len(message_codé)} bits, soit " +
      f"{len(message_codé)/8} octets.")
print(10*"---")

# Partie décodage :

message_décodé = decode_message(message_codé, arbre)
print(f"Le message décodé est : \n{message_décodé}")


Le message codé est :
11011001101101000101010010010101101100111110111101011011001111011110001000010010000101011100011100100100111101111001110101000100110111000011100101001110111111111100010011110011100010011001001100110101111010010111111111101000000100000011000110011010
------------------------------
La taille du message codé est de : 248 bits, soit 31.0 octets.
------------------------------
Le message décodé est : 
Voici quelques caractères à coder :

ab€d €bbc
cc

4! = 24
